# Data Cleaning
- Find major cities
- Filter by category


In [1]:
import lightfm
import scipy as sp
import pymongo
import pandas as pd
import numpy as np

c:\Users\Cody\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [6]:
# Read in data
# Connect to mongodb on 192.168.1.87
client = pymongo.MongoClient(
    host='192.168.1.87',
    port=27017)
# Get the yelp database
db = client.yelp
review = db.review
business = db.business
user = db.user

In [9]:
# Get count of business by city and state
business_by_city = business.aggregate(
    [
        {
            '$group': {
                '_id': {
                    'city': '$city',
                    'state': '$state'
                },
                'count': {
                    '$sum': 1
                }
            }
        }, {
            '$sort': {
                'count': -1
            }
        }
    ])
business_by_city = pd.DataFrame(list(business_by_city))

In [11]:
business_by_city.head(10)

,_id,count
0,"{'city': 'Philadelphia', 'state': 'PA'}",14567
1,"{'city': 'Tucson', 'state': 'AZ'}",9249
2,"{'city': 'Tampa', 'state': 'FL'}",9048
3,"{'city': 'Indianapolis', 'state': 'IN'}",7540
4,"{'city': 'Nashville', 'state': 'TN'}",6968
5,"{'city': 'New Orleans', 'state': 'LA'}",6208
6,"{'city': 'Reno', 'state': 'NV'}",5932
7,"{'city': 'Edmonton', 'state': 'AB'}",5054
8,"{'city': 'Saint Louis', 'state': 'MO'}",4827
9,"{'city': 'Santa Barbara', 'state': 'CA'}",3829


In [15]:
# Check business types
# Split "categories" column into list based on "," delim
business_cats = business.aggregate([
    {
        '$project': {
            "categories": {"$split": ["$categories", ", "]}
        }
    },
    {
        '$unwind': {
            'path': '$categories'
        }
    }, {
        '$group': {
            '_id': '$categories',
            'count': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'count': -1
        }
    }
])
businesses_by_category = pd.DataFrame(list(business_cats))
businesses_by_category.head(20)

,_id,count
0,Restaurants,52268
1,Food,27781
2,Shopping,24395
3,Home Services,14356
4,Beauty & Spas,14292
5,Nightlife,12281
6,Health & Medical,11890
7,Local Services,11198
8,Bars,11065
9,Automotive,10773


In [ ]:
keep_categories = [
    'Restaurants',
    'Food',
    'Shopping',
    'Beauty & Spas',
    'Nightlife',
    'Bars',
    'Sandwiches',
    'American (Traditional)',
    'Pizza',
    'Coffee & Tea',
    'Fast Food',
    'Breakfast & Brunch	',
    'American (New)',
    ]
